In [1]:
import numpy as np
import itk
import os

In [2]:
# Set path to test files and import
chosen_im = 'copd2'

# Set path to data folder containing copd1, copd2, etc. folders
data_dir = "../data"

#Set paths to landmarks and images
path_landmarks1 = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_eBH_xyz_r1.txt")
path_landmarks2 = os.path.join(data_dir, f"{chosen_im}/{chosen_im}_300_iBH_xyz_r1.txt")
path_img1 = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_eBHCT.mha')
path_img2 = os.path.join(data_dir, f'./{chosen_im}/{chosen_im}_iBHCT.mha')

In [3]:
landmarks1 = np.loadtxt(path_landmarks1)
landmarks2 = np.loadtxt(path_landmarks2)

pixel_type = itk.ctype('unsigned short')
img1 = itk.imread(path_img1, pixel_type)

In [5]:
# Function to calculate TRE
def TRE(landmarks1, landmarks2, spacing):
    """
    mean (and standard deviation) 3D Euclidean magnitude distance 
    between calculated and reference landmark positions for the 
    set of validation landmarks. All values are reported in units of millimeters.
    
    Parameters:    
        landmarks1 (ndarry): first set of landmarks.
        landmarks2 (ndarry): second set of landmarks.
        spaing (tuple(float)): spacing for x, y, z.
    
    Returns:
        (floar, float) mean and SD 3D Euclidean magnitude distance.
    """
    landmarks1 = spacing*landmarks1
    landmarks2 = spacing*landmarks2
    diff = landmarks1 - landmarks2
    squared = diff * diff
    dist = np.sqrt(np.sum(squared,axis=1))
    mean_TRE = np.mean(dist)
    sd_TRE = np.std(dist)

    return dist, mean_TRE, sd_TRE

In [6]:
dist, mean_TRE, SD_TRE = TRE(landmarks1, landmarks2, tuple(img1.GetSpacing()))
print(mean_TRE, SD_TRE)

21.64084010925038 6.415267676867441


In [25]:
help(itk.ImageRegistrationMethodv4.New())

Help on itkImageRegistrationMethodv4REGv4F2F2 in module itkImageRegistrationMethodv4Python object:

class itkImageRegistrationMethodv4REGv4F2F2(ITKCommonBasePython.itkProcessObject)
 |  Interface method for the current registration framework.
 |  
 |  This interface method class encapsulates typical registration usage by
 |  incorporating all the necessary elements for performing a simple image
 |  registration between two images. This method also allows for
 |  multistage registration whereby each stage is characterize by possibly
 |  different transforms of and different a linear registration followed
 |  by deformable registration where both stages are performed in multiple
 |  levels. Each level can be characterized by:
 |  
 |  the resolution of the virtual domain image (see below)
 |  
 |  smoothing of the fixed and moving images
 |  
 |  the coarseness of the current transform via transform adaptors (see
 |  below)  Multiple stages are handled by linking multiple instantiations


In [27]:
FixedImageType   = itk.Image[float, 3]
MovingImageType  = itk.Image[float, 3]
TransformType    = itk.Rigid3DTransform[itk.D]
OptimizerType    = itk.RegularStepGradientDescentOptimizerv4[itk.D]
RegistrationType = itk.ImageRegistrationMethodv4[FixedImageType,
                                                 MovingImageType]
MetricType       = itk.MeanSquaresImageToImageMetricv4[FixedImageType,
                                                       MovingImageType, TransformType]

KeyError: "itkTemplate : No template (<class 'float'>, 3) for the itk::Image class"

In [23]:

#
#  Read the fixed and moving images using filenames
#  from the command line arguments
#
fixedImageReader  = itk.ImageFileReader[FixedImageType].New()
movingImageReader = itk.ImageFileReader[MovingImageType].New()

fixedImageReader.SetFileName(path_img1)
movingImageReader.SetFileName(path_img2)

fixedImageReader.Update()
movingImageReader.Update()

fixedImage  = fixedImageReader.GetOutput()
movingImage = movingImageReader.GetOutput()


#
#  Instantiate the classes for the registration framework
#
registration = RegistrationType.New()
imageMetric  = MetricType.New()
transform    = TransformType.New()
optimizer    = OptimizerType.New()

registration.SetOptimizer(optimizer)
registration.SetMetric(imageMetric)

registration.SetFixedImage(fixedImage)
registration.SetMovingImage(movingImage)


#
# Initial transform parameters
#
transform.SetAngle( 0.0 )

# center of the fixed image
fixedSpacing = fixedImage.GetSpacing()
fixedOrigin = fixedImage.GetOrigin()
fixedSize = fixedImage.GetLargestPossibleRegion().GetSize()

centerFixed = ( fixedOrigin.GetElement(0) + fixedSpacing.GetElement(0) * fixedSize.GetElement(0) / 2.0,
                fixedOrigin.GetElement(1) + fixedSpacing.GetElement(1) * fixedSize.GetElement(1) / 2.0 )

# center of the moving image
movingSpacing = movingImage.GetSpacing()
movingOrigin = movingImage.GetOrigin()
movingSize = movingImage.GetLargestPossibleRegion().GetSize()

centerMoving = ( movingOrigin.GetElement(0) + movingSpacing.GetElement(0) * movingSize.GetElement(0) / 2.0,
                 movingOrigin.GetElement(1) + movingSpacing.GetElement(1) * movingSize.GetElement(1) / 2.0  )

# transform center
center = transform.GetCenter()
center.SetElement( 0, centerFixed[0] )
center.SetElement( 1, centerFixed[1] )

# transform translation
translation = transform.GetTranslation()
translation.SetElement( 0, centerMoving[0] - centerFixed[0] )
translation.SetElement( 1, centerMoving[1] - centerFixed[1] )

registration.SetInitialTransform(transform)

initialParameters = transform.GetParameters()

print("Initial Parameters: ")
print("Angle: %f" % (initialParameters.GetElement(0),))
print("Center: %f, %f" % ( initialParameters.GetElement(1), initialParameters.GetElement(2)))
print("Translation: %f, %f" % (initialParameters.GetElement(3), initialParameters.GetElement(4)))


#
# Define optimizer parameters
#

# optimizer scale
translationScale = 1.0 / 1000.0

optimizerScales = itk.OptimizerParameters[itk.D](transform.GetNumberOfParameters())
optimizerScales.SetElement(0, 1.0)
optimizerScales.SetElement(1, translationScale)
optimizerScales.SetElement(2, translationScale)
optimizerScales.SetElement(3, translationScale)
optimizerScales.SetElement(4, translationScale)

optimizer.SetScales( optimizerScales )

optimizer.SetRelaxationFactor( 0.6 );
optimizer.SetLearningRate( 0.1 );
optimizer.SetMinimumStepLength( 0.001 );
optimizer.SetNumberOfIterations( 200 );


#
# One level registration process without shrinking and smoothing.
#
registration.SetNumberOfLevels(1)
registration.SetSmoothingSigmasPerLevel([0])
registration.SetShrinkFactorsPerLevel([1])


#
# Iteration Observer
#
def iterationUpdate():
    currentParameter = transform.GetParameters()
    print("M: %f   P: %f %f %f %f %f " % ( optimizer.GetValue(),
                                 currentParameter.GetElement(0),
                                 currentParameter.GetElement(1),
                                 currentParameter.GetElement(2),
                                 currentParameter.GetElement(3),
                                 currentParameter.GetElement(4)))

iterationCommand = itk.PyCommand.New()
iterationCommand.SetCommandCallable( iterationUpdate )
optimizer.AddObserver( itk.IterationEvent(), iterationCommand )

print("Starting registration")


#
# Start the registration process
#
registration.Update()


#
# Get the final parameters of the transformation
#
finalParameters = registration.GetOutput().Get().GetParameters()

print("Final Registration Parameters ")
print("Angle in radians  = %f" % finalParameters.GetElement(0))
print("Rotation Center X = %f" % finalParameters.GetElement(1))
print("Rotation Center Y = %f" % finalParameters.GetElement(2))
print("Translation in  X = %f" % finalParameters.GetElement(3))
print("Translation in  Y = %f" % finalParameters.GetElement(4))


#
# Now, we use the final transform for resampling the
# moving image.
#
resampler = itk.ResampleImageFilter[MovingImageType,FixedImageType].New()
resampler.SetTransform(registration.GetTransform())
resampler.SetInput(movingImageReader.GetOutput())

region = fixedImage.GetLargestPossibleRegion()

resampler.SetSize(region.GetSize())
resampler.SetOutputOrigin(fixedImage.GetOrigin())
resampler.SetOutputSpacing(fixedImage.GetSpacing())
resampler.SetOutputDirection(fixedImage.GetDirection())
resampler.SetDefaultPixelValue(100)

OutputImageType  = itk.Image[itk.UC, 2]
outputCast = itk.CastImageFilter[FixedImageType, OutputImageType].New()
outputCast.SetInput(resampler.GetOutput())


#
#  Write the resampled image
#
writer = itk.ImageFileWriter[OutputImageType].New()
writer.SetFileName( argv[3] )
writer.SetInput( outputCast.GetOutput() )
writer.Update()

KeyError: "itkTemplate : No template (<class 'itkImagePython.itkImageUS3'>, <class 'itkImagePython.itkImageUS3'>) for the itk::ImageRegistrationMethodv4 class"